شروع و دریافت  اطلاعات

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from keras.layers import LSTM, Dense, Input, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Input
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from sklearn.model_selection import KFold

In [ ]:
# تنظیمات
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_H1
number_of_candles = 99999

mt5.initialize()
# login = '2424394'
# password = 'Aa@12345678'
# server = 'AMarkets-Real'
# mt5.login(login, password, server)


def get_data(symbol, timeframe, number_of_candles):
    if not mt5.initialize():
        print("Failed to initialize MT5")
        return None

    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, number_of_candles)
    mt5.shutdown()

    candles_data = pd.DataFrame(rates)
    candles_data['time'] = pd.to_datetime(candles_data['time'], unit='s')
    return candles_data


candles = get_data(symbol, timeframe, number_of_candles)

# تغییر نام
candles = candles.rename(
    columns={'open': 'O', 'high': 'H', 'low': 'L', 'close': 'C', 'tick_volume': 'vol'})
candles = candles.drop(columns=['real_volume', 'spread'])


candles

پیدا کردن فراکتال ها

In [ ]:
# استفاده از تابع fractals
left_n = 3  # تعداد کندل‌های چپ
right_n = 2  # تعداد کندل‌های راست

# تعریف تابع fractals


def fractal(candles, left_n, right_n):
    # ایجاد ستون‌های اولیه برای فراکتال‌ها با مقدار False
    candles['up_f'] = True
    candles['low_f'] = True

    # شناسایی فراکتال‌های بالا
    for i in range(1, left_n + 1):
        candles['up_f'] &= (candles['H'] >= candles['H'].shift(i))

    for j in range(1, right_n + 1):
        candles['up_f'] &= (candles['H'] >= candles['H'].shift(-j))

    # شناسایی فراکتال‌های پایین
    for i in range(1, left_n + 1):
        candles['low_f'] &= (candles['L'] <= candles['L'].shift(i))

    for j in range(1, right_n + 1):
        candles['low_f'] &= (candles['L'] <= candles['L'].shift(-j))

    # تبدیل فراکتال‌های بالا و پایین به مقادیر بولی
    candles['up_f'] = np.where(candles['up_f'], True, False)
    candles['low_f'] = np.where(candles['low_f'], True, False)

    return candles


# فراخوانی تابع fractals برای شناسایی فراکتال‌ها
fractals = fractal(candles, left_n, right_n)


# نمایش نتیجه
fractals

In [ ]:
# گنترل اینکه فراکتال هست یا نه
fractals['fra'] = fractals.apply(
    lambda row: 'True' if row['up_f'] or row['low_f'] else 'False', axis=1)

fractals[-50:]

حذف فراکتال های پشت سر هم

In [ ]:

fractals['frac'] = False  # ستون جدید برای اکستریم‌ها، پیش‌فرض False

# لیست نهایی اندیس‌هایی که اکستریم هستند
final_rows = []

# لیست‌های موقت برای ذخیره اندیس‌های up_f و low_f
temp_up_f = []  # اندیس‌های up_f
temp_low_f = []  # اندیس‌های low_f

# تابع عمومی برای پیدا کردن اکستریم


def process_fracs(df, frac_list, column, frac_type):
    if len(frac_list) > 0:
        frac_index = find_frac(df, frac_list, column, frac_type)
        final_rows.append(frac_index)
        frac_list.clear()  # خالی کردن لیست موقت

# تابع برای پیدا کردن اکستریم


def find_frac(df, indexes, column, frac_type='max'):
    if frac_type == 'max':
        return df.loc[indexes, column].idxmax()
    elif frac_type == 'min':
        return df.loc[indexes, column].idxmin()


# پردازش ردیف‌ها
for i, row in fractals.iterrows():
    if row['fra']:  # فقط ردیف‌هایی که fra برابر True دارند
        if row['up_f']:  # اگر up_f برابر True باشد
            temp_up_f.append(i)  # ذخیره اندیس up_f
            process_fracs(fractals, temp_low_f, 'L', 'min')
        elif row['low_f']:  # اگر low_f برابر True باشد
            temp_low_f.append(i)  # ذخیره اندیس low_f
            process_fracs(fractals, temp_up_f, 'H', 'max')

# اضافه کردن آخرین اکستریم‌ها
if len(temp_up_f) > 0:
    final_rows.append(find_frac(fractals, temp_up_f, 'H', 'max'))

if len(temp_low_f) > 0:
    final_rows.append(find_frac(fractals, temp_low_f, 'L', 'min'))

# علامت‌گذاری اکستریم‌ها در ستون frac
fractals['frac'] = fractals.index.isin(final_rows)


# نمایش دیتافریم نهایی
fractals[-350:-300]

جاهایی که یک کندل هم فراکتال سقف هست هم فراکتال کف

In [ ]:
# پیمایش دیتافریم و چاپ شماره ردیف‌هایی که هر دو ستون up_f و low_f برابر True هستند
for i, row in fractals.iterrows():
    if row['up_f'] and row['low_f']:
        print(f"Row {i}: Both up_f and low_f are True")

        # پیدا کردن اولین ردیف به سمت بالا که low_f یا up_f برابر True باشد
        upper_index = i - 1
        while upper_index >= 0:  # اطمینان از اینکه به ردیف‌های منفی نرسیم
            if fractals.iloc[upper_index]['low_f']:
                print(f"First upper Row {upper_index}: low_f is True")
                break
            elif fractals.iloc[upper_index]['up_f']:
                print(f"First upper Row {upper_index}: up_f is True")
                break
            upper_index -= 1

        # پیدا کردن اولین ردیف به سمت پایین که low_f یا up_f برابر True باشد
        lower_index = i + 1
        # اطمینان از اینکه به ردیف‌های بیش از طول نرسیم
        while lower_index < len(fractals):
            if fractals.iloc[lower_index]['up_f']:
                print(f"First lower Row {lower_index}: up_f is True")
                break
            elif fractals.iloc[lower_index]['low_f']:
                print(f"First lower Row {lower_index}: low_f is True")
                break
            lower_index += 1

fractals[620:670]

حذف یکی از طرفین بالا یا پایین کندل هایی که فراکتال بالا پایین هستند با توجه به فراکتال قبلی

In [ ]:
# پیمایش دیتافریم و چاپ شماره ردیف‌هایی که هر دو ستون up_f و low_f برابر True هستند
for i, row in fractals.iterrows():
    if row['up_f'] and row['low_f']:
        print(f"Row {i}: Both up_f and low_f are True")

        # پیدا کردن اولین ردیف به سمت بالا که low_f یا up_f برابر True باشد
        upper_index = i - 1
        while upper_index >= 0:  # اطمینان از اینکه به ردیف‌های منفی نرسیم
            if fractals.iloc[upper_index]['low_f']:
                print(f"First upper Row {upper_index}: low_f is True")
                # اگر فراکتال قبلی low_f باشد، low_f فعلی را False کنیم
                fractals.at[i, 'low_f'] = False
                # پرینت جایی که low_f به False تنظیم شده است
                print(f"Row {i}: low_f set to False")
                break
            elif fractals.iloc[upper_index]['up_f']:
                print(f"First upper Row {upper_index}: up_f is True")
                # اگر فراکتال قبلی up_f باشد، up_f فعلی را False کنیم
                fractals.at[i, 'up_f'] = False
                # پرینت جایی که up_f به False تنظیم شده است
                print(f"Row {i}: up_f set to False")
                break
            upper_index -= 1

# نمایش بخشی از دیتافریم برای بررسی تغییرات
fractals[476:483]

مشخص کردن مقادیر لگ ها

In [ ]:
# مشخص کردن مقادیر لگ ها
fractals['leg_p'] = None

# پیمایش دیتافریم و ثبت مقادیر H و L در ستون‌های جدید
for i, row in fractals.iterrows():
    if row['frac']:  # اگر frac برابر True باشد
        if row['up_f']:  # اگر فقط up_f ترو باشد
            fractals.at[i, 'leg_p'] = row['H']  # نوشتن مقدار H
        elif row['low_f']:  # اگر فقط low_f ترو باشد
            fractals.at[i, 'leg_p'] = row['L']  # نوشتن مقدار L


# نمایش دیتا فریم نهای
fractals[-50:]

In [ ]:
# انتخاب ردیف‌های خاص از دیتافریم
subset_fractals = fractals.iloc[-500:]  # انتخاب ردیف‌های 10 تا 20
# حذف سطرهایی که مقدار 'fra' آن‌ها False است
filtered_fractals = subset_fractals[subset_fractals['frac']]

# رسم نمودار
plt.figure(figsize=(12, 6))

# رسم نقاط up_f به رنگ آبی
plt.plot(filtered_fractals['time'][filtered_fractals['up_f']], filtered_fractals['leg_p']
         [filtered_fractals['up_f']], 'o', markersize=4, color='blue', label='up_f')

# رسم نقاط low_f به رنگ قرمز
plt.plot(filtered_fractals['time'][filtered_fractals['low_f']], filtered_fractals['leg_p']
         [filtered_fractals['low_f']], 'o', markersize=4, color='red', label='low_f')

# رسم نمودار خطی برای leg_p و اتصال نقاط
plt.plot(filtered_fractals['time'], filtered_fractals['leg_p'],
         '-', color='green', label='leg_p')

plt.xlabel('Time')
plt.ylabel('leg_p')
plt.title('Time Series of leg_p (Filtered Rows)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

# پیمایش معکوس ستون leg_p و جایگزینی مقادیر None
previous_value = None
for i in range(len(fractals)-1, -1, -1):
    if fractals.loc[i, 'leg_p'] is not None:
        previous_value = fractals.loc[i, 'leg_p']
    else:
        fractals.loc[i, 'leg_p'] = previous_value

fractals[-50:]

شروع فرایند یادگیری ماشین

In [ ]:
# به دلیل وجود None و اینکه مقدار صفر مناسب است و باعث ایجاد اشتباه نمیشود با صفر جایگزین شد
fractals['leg_p'] = fractals['leg_p'].fillna(0)

# تبدیل نوع ستون به نوع مناسب
fractals['leg_p'] = fractals['leg_p'].infer_objects(copy=False)

fractals

In [ ]:
fractals_data = fractals
fractals_data

نرمال سازی داده ها

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# در صورت تغییر به feature_range مورد نظر
scaler = MinMaxScaler(feature_range=(0, 1))

# انتخاب ستون‌هایی که می‌خواهیم نرمال کنیم
features_to_scale = ['O', 'H', 'L', 'C', 'vol', 'leg_p']

# اعمال MinMaxScaler به ستون‌های مورد نظر
fractals[features_to_scale] = scaler.fit_transform(fractals[features_to_scale])

# نمایش دیتا فریم نرمال شده
fractals[-50:]

ساخت مدل RNN

ترین و تست 

روش 1- ترین و تست به روش تقسیم ساده

In [ ]:
# آماده‌سازی داده‌ها
sequence_length = 1
features = ['O', 'H', 'L', 'C', 'vol']
X = np.array([fractals[features].iloc[i:i +
             sequence_length].values for i in range(len(fractals) - sequence_length)])
X = X.astype(np.float32)  # تبدیل نوع داده‌ها به float32
y = fractals['leg_p'].iloc[sequence_length:].astype(
    np.float32).values  # تبدیل y به float32

# تقسیم داده‌ها به مجموعه‌های آموزشی و تست
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# ساخت مدل
model = Sequential()
model.add(Input(shape=(sequence_length, len(features))))
model.add(LSTM(4, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(LSTM(2))
model.add(Dense(1))

# کامپایل مدل
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# تنظیمات توقف زودهنگام و کاهش نرخ یادگیری
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

# آموزش مدل
history = model.fit(X_train, y_train, validation_data=(
    X_test, y_test), epochs=1, batch_size=32, callbacks=[early_stopping, reduce_lr])

# پیش‌بینی روی مجموعه تست
predictions = model.predict(X_test)

# محاسبه خطای مطلق میانگین
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

# ترسیم نمودار مقایسه
plt.figure(figsize=(10, 6))
plt.plot(y_test[-100:], label='Actual')
plt.plot(predictions[-100:], label='Predicted')
plt.title('Comparison of Actual and Predicted Values (Last 100 Points)')
plt.xlabel('Sample')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:


# فرض کنیم که شما در زمان نرمال‌سازی از 6 ویژگی استفاده کردید.
# بنابراین X_test باید تنها شامل ویژگی‌های مورد استفاده باشد.

# تغییر شکل X_test به ابعاد صحیح
X_test_full = np.concatenate([X_test.reshape(
    # ترکیب ویژگی‌ها و y_test
    X_test.shape[0], -1), y_test.reshape(-1, 1)], axis=1)

# تأیید اینکه تعداد ستون‌ها با تعداد ویژگی‌های نرمال‌سازی شده مطابقت دارد
print(f"Shape of X_test_full: {X_test_full.shape}")  # بررسی شکل

# نرمال‌سازی معکوس y_test
y_test_scaled_back = scaler.inverse_transform(
    X_test_full[:, :6])[:, -1]  # تنها ویژگی‌های اولیه را معکوس کنید

# نرمال‌سازی معکوس predictions
predictions_full = np.concatenate([X_test.reshape(
    X_test.shape[0], -1), predictions], axis=1)  # تغییر شکل برای ترکیب
predictions_scaled_back = scaler.inverse_transform(predictions_full[:, :6])[
    :, -1]  # تنها ویژگی‌های اولیه را معکوس کنید

# نمایش نتایج برای 5 ردیف آخر
print("\nLast 5 Predictions:")
for i in range(-5, 0):
    print(f'Actual (original scale): {
          y_test_scaled_back[i]}, Predicted (original scale): {predictions_scaled_back[i]}')

In [ ]:
# فرض کنید predictions یک آرایه از پیش‌بینی‌های شما باشد
predictions = predictions_scaled_back  # پیش‌بینی‌ها را از مرحله قبل گرفته‌ایم

# ایجاد یک ستون جدید برای پیش‌بینی‌ها
fractals['predictions'] = None  # ایجاد ستون جدید با مقادیر None

# ایندکس شروع برای پیش‌بینی‌ها
start_index = 100  # تنظیم ایندکس شروع بر اساس نیاز شما
predicted_index = 0  # ایندکس پیش‌بینی برای پیمایش در predictions

# تخصیص مقادیر پیش‌بینی‌شده به مکان‌های مربوطه
for i in range(start_index, start_index + len(predictions) * repeat_count, repeat_count):
    if predicted_index < len(predictions):
        fractals.loc[i:i + repeat_count - 1,
                     'predictions'] = predictions[predicted_index]
        predicted_index += 1

# نمایش دیتافریم جدید
print(fractals.tail(100))  # نمایش 50 ردیف آخر برای تأیید

روش 2- ترین و تست به روش کراس ولیدیشن

In [ ]:
# آماده‌سازی داده‌ها
sequence_length = 20
features = ['O', 'H', 'L', 'C', 'vol']
X = np.array([fractals[features].iloc[i:i +
             sequence_length].values for i in range(len(fractals) - sequence_length)])
X = X.astype(np.float32)  # تبدیل نوع داده‌ها به float32
y = fractals['leg_p'].iloc[sequence_length:].astype(
    np.float32).values  # تبدیل y به float32

# تعریف Cross-Validation
kf = KFold(n_splits=3)
fold = 1

for train_index, test_index in kf.split(X):
    print(f'Fold {fold}')

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # ساخت مدل
    model = Sequential()
    model.add(Input(shape=(sequence_length, len(features))))
    model.add(LSTM(50, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(20))
    model.add(Dense(1))

    # کامپایل مدل
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error')

    # تنظیمات توقف زودهنگام و کاهش نرخ یادگیری
    early_stopping = EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

    # آموزش مدل
    history = model.fit(X_train, y_train, validation_data=(
        X_test, y_test), epochs=10, batch_size=32, callbacks=[early_stopping, reduce_lr])

    # پیش‌بینی روی مجموعه تست
    predictions = model.predict(X_test)

    # ترسیم نمودار مقایسه
    plt.figure(figsize=(10, 6))
    plt.plot(y_test_scaled_back[-100:-10], label='Actual')
    plt.plot(predictions_scaled_back[-100:], label='Predicted')
    plt.title(f'Comparison of Actual and Predicted Values for Fold {
              fold} (Last 100 Points)')
    plt.xlabel('Sample')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

    fold += 1

In [ ]:
# نمایش 5 نتیجه آخر
print("\nLast 5 Predictions:")
for i in range(-10, 0):
    print(f'Actual (original scale): {
          y_test_scaled_back[i]}, Predicted (original scale): {predictions_scaled_back[i]}')